In [70]:
import os
import ast
import shutil
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [87]:
payload='email=XXXXXXX%40XXXXXXX&password=XXXXXXX'

In [88]:
headers = {
  'cookie': 'XXXXXXX',
  'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Mobile Safari/537.36',
  'Content-Type': 'application/x-www-form-urlencoded'
}

In [89]:
def get_soup(url):
    response = requests.request("GET", url, headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [90]:
df = pd.read_csv("csv_files/df_all_pages.csv")
df.head()

,page_lesson,page_name,page_id,page_number
0,Lesson 1,Welcome to Web Development!,283705,Page 1
1,Lesson 1,The Future of Web Development,283706,Page 2
2,Lesson 1,What is Python?,283707,Page 3
3,Lesson 1,Course Goals,283708,Page 4
4,Lesson 2,Lesson 11 - Variables and Datatypes,283703,Page 1


In [91]:
problematic_pages = ['/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_12/page_1.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_12/page_4.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_15/page_1.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_15/page_3.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_15/page_5.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_14/page_2.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_13/page_1.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_13/page_3.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_16/page_2.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_11/page_2.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_11/page_4.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_17/page_1.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_4/page_7.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_4/page_3.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_4/page_5.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_4/page_9.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_3/page_7.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_3/page_5.html',
 '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_5/page_2.html']

In [92]:
def get_tups(file_path):
    file_path = file_path.split("/Users/vivianamarquez/Desktop/prisoneducation/raw_html/")[1]
    tokens = file_path[:-5].split("/")
    tokens = [token.capitalize().replace("_", " ") for token in tokens]
    return tokens

In [93]:
problematic_pages = [get_tups(p) for p in problematic_pages]

In [94]:
df.set_index(['page_lesson', 'page_number'], inplace=True)

In [95]:
ids_to_scrape = [df.loc[(p[0], p[1]), 'page_id'] for p in problematic_pages]

In [96]:
combined_list = [list(item) + [id_] for item, id_ in zip(problematic_pages, ids_to_scrape)]

In [97]:
combined_list

[['Lesson 12', 'Page 1', 283673],
 ['Lesson 12', 'Page 4', 283676],
 ['Lesson 15', 'Page 1', 283685],
 ['Lesson 15', 'Page 3', 283687],
 ['Lesson 15', 'Page 5', 283689],
 ['Lesson 14', 'Page 2', 283682],
 ['Lesson 13', 'Page 1', 283678],
 ['Lesson 13', 'Page 3', 283680],
 ['Lesson 16', 'Page 2', 283691],
 ['Lesson 11', 'Page 2', 283669],
 ['Lesson 11', 'Page 4', 283671],
 ['Lesson 17', 'Page 1', 283694],
 ['Lesson 4', 'Page 7', 283722],
 ['Lesson 4', 'Page 3', 283718],
 ['Lesson 4', 'Page 5', 283720],
 ['Lesson 4', 'Page 9', 283724],
 ['Lesson 3', 'Page 7', 283715],
 ['Lesson 3', 'Page 5', 283713],
 ['Lesson 5', 'Page 2', 283726]]

In [98]:
def save_html(url_id, lesson, page):
    try:
        lesson_folder_name = lesson.lower().replace(" ", "_")
        page_folder_name = page.lower().replace(" ", "_")
        os.makedirs(f"raw_html/{lesson_folder_name}", exist_ok=True)

        page_url = f"https://courses.edovo.com/lesson/get_page_items?pageId={url_id}"
        page_soup = get_soup(page_url)
        html_str = page_soup.prettify()

        with open(f'raw_html/{lesson_folder_name}/{page_folder_name}.html', 'w', encoding='utf-8') as file:
            file.write(html_str)

        print(f"{lesson} - {page} completed.")
    
    except:
        print(f"ISSUE with {lesson} - {page}.")

In [99]:
for lesson, page, url_id in combined_list:
    save_html(url_id, lesson, page)

Lesson 12 - Page 1 completed.
Lesson 12 - Page 4 completed.
Lesson 15 - Page 1 completed.
Lesson 15 - Page 3 completed.
Lesson 15 - Page 5 completed.
Lesson 14 - Page 2 completed.
Lesson 13 - Page 1 completed.
Lesson 13 - Page 3 completed.
Lesson 16 - Page 2 completed.
Lesson 11 - Page 2 completed.
Lesson 11 - Page 4 completed.
Lesson 17 - Page 1 completed.
Lesson 4 - Page 7 completed.
Lesson 4 - Page 3 completed.
Lesson 4 - Page 5 completed.
Lesson 4 - Page 9 completed.
Lesson 3 - Page 7 completed.
Lesson 3 - Page 5 completed.
Lesson 5 - Page 2 completed.
